# Open Route Service

[**Open Route Service**](https://openrouteservice.org/) is a public geo-data API service. It can generate directions and routes between locations, offering a wide range of transport options (e.g. bike, car, walking, etc.) A [Python wrapper](https://anaconda.org/MichaelsJP/openrouteservice) can be installed to take care of the API requests. Using Anaconda Navigator, you will need to click the green arrow on your project environment, "Open Terminal", and in the terminal, paste:

```shell
conda install michaelsjp::openrouteservice
```

Why do we need this? Well, if we simply try to connect two locations using a simple PolyLine, we will have an "as the crow flies" indication of distance between two coordinates, but no information about the roads and actual travel distance. 

In [10]:
import folium

f = folium.Figure(width=700, height=500)
m = folium.Map(location=[52.5300, 13.4150], zoom_start=13).add_to(f)

start_coord = (52.5442751, 13.4408026)
end_coord = (52.5162699, 13.3777034)

folium.Marker(start_coord, tooltip="Code Academy Berlin").add_to(m)
folium.Marker(end_coord, tooltip="Brandenburger Tor").add_to(m)

folium.PolyLine([
    start_coord,
    end_coord,
]).add_to(m)

m

Open Route Service is a free API, but you will still need to sign up for an API key in order to use their service. Simply create an account, and you can find your API key in your profile. API keys should be kept private! If you use this service, do not push your API key to GitHub, use **environment variables**. Use your API key to initialize a "client" instance. We will use this to make requests to the API on our behalf for information.

In [11]:
import openrouteservice
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("OPENROUTESERVICE_APIKEY")
client = openrouteservice.Client(key=API_KEY)

Let's see how much Open Route Service knows about our locations (**note**: you'll need to reverse the order of latitude/longitude for ORS):

In [13]:
start_coords_ors = (start_coord[1], start_coord[0])
end_coords_ors = (end_coord[1], end_coord[0])

start_location = client.pelias_reverse(point=start_coords_ors)['features'][0]['properties']['label']
end_location = client.pelias_reverse(point=end_coords_ors)['features'][0]['properties']['label']
print("start:", start_location, "\nend:  ", end_location)

start: Erich-Weinert-Straße 145, Berlin, Germany 
end:   Brandenburger Tor, Berlin, Germany


In [14]:
import json

print(json.dumps(client.pelias_reverse(point=start_coords_ors), indent=2))

{
  "geocoding": {
    "version": "0.2",
    "attribution": "https://openrouteservice.org/terms-of-service/#attribution-geocode",
    "query": {
      "size": 10,
      "private": false,
      "point.lat": 52.544275,
      "point.lon": 13.440803,
      "boundary.circle.lat": 52.544275,
      "boundary.circle.lon": 13.440803,
      "lang": {
        "name": "English",
        "iso6391": "en",
        "iso6393": "eng",
        "via": "default",
        "defaulted": true
      },
      "querySize": 20
    },
    "engine": {
      "name": "Pelias",
      "author": "Mapzen",
      "version": "1.0"
    },
    "timestamp": 1742294157175
  },
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [
          13.440792,
          52.544288
        ]
      },
      "properties": {
        "id": "way/22062177",
        "gid": "openstreetmap:address:way/22062177",
        "layer": "address",
        "source"

Get **cycling route** between the two points:

In [15]:
# Get cycling route between the two points
routes = client.directions(
    coordinates=[start_coords_ors, end_coords_ors],
    profile='cycling-regular',
    format='geojson'
)

# Extract the route geometry
route_coords_ors = routes['features'][0]['geometry']['coordinates']
route_coords_ors

[[13.441046, 52.544185],
 [13.440983, 52.544122],
 [13.440883, 52.544036],
 [13.440768, 52.544013],
 [13.44053, 52.543779],
 [13.440466, 52.543714],
 [13.440421, 52.543729],
 [13.43981, 52.543939],
 [13.439259, 52.544128],
 [13.438715, 52.544315],
 [13.437937, 52.544583],
 [13.437842, 52.544616],
 [13.437763, 52.544644],
 [13.437512, 52.544727],
 [13.436946, 52.544928],
 [13.436846, 52.544962],
 [13.43673, 52.545003],
 [13.436313, 52.545148],
 [13.435323, 52.545492],
 [13.434292, 52.545845],
 [13.434194, 52.545879],
 [13.434094, 52.545913],
 [13.433211, 52.546224],
 [13.433117, 52.546256],
 [13.433031, 52.546287],
 [13.432231, 52.546565],
 [13.432164, 52.546588],
 [13.432092, 52.546612],
 [13.431268, 52.546897],
 [13.431198, 52.54692],
 [13.431142, 52.546863],
 [13.430129, 52.545815],
 [13.43009, 52.545776],
 [13.430018, 52.545807],
 [13.429661, 52.545962],
 [13.429562, 52.546002],
 [13.429398, 52.54601],
 [13.428696, 52.545436],
 [13.428631, 52.54538],
 [13.428054, 52.545636],
 [13.42

In [18]:
routes

{'type': 'FeatureCollection',
 'features': [{'bbox': [13.37772, 52.51616, 13.441046, 52.54692],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 6783.9,
      'duration': 1476.6,
      'steps': [{'distance': 67.2,
        'duration': 20.7,
        'type': 11,
        'instruction': 'Head southwest',
        'name': '-',
        'way_points': [0, 5]},
       {'distance': 721.1,
        'duration': 144.2,
        'type': 1,
        'instruction': 'Turn right onto Erich-Weinert-Straße',
        'name': 'Erich-Weinert-Straße',
        'way_points': [5, 29]},
       {'distance': 147.7,
        'duration': 29.5,
        'type': 0,
        'instruction': 'Turn left onto Sültstraße',
        'name': 'Sültstraße',
        'way_points': [29, 32]},
       {'distance': 54.8,
        'duration': 11.0,
        'type': 1,
        'instruction': 'Turn right onto Küselstraße',
        'name': 'Küselstraße',
        'way_points': [32, 36]},
       {'distance': 87.2,
        'duration': 

In [16]:
# Convert [lon, lat] to (lat, lon) for Folium using Python List comprehension
route_coords = [(coord[1], coord[0]) for coord in route_coords_ors]
route_coords

[(52.544185, 13.441046),
 (52.544122, 13.440983),
 (52.544036, 13.440883),
 (52.544013, 13.440768),
 (52.543779, 13.44053),
 (52.543714, 13.440466),
 (52.543729, 13.440421),
 (52.543939, 13.43981),
 (52.544128, 13.439259),
 (52.544315, 13.438715),
 (52.544583, 13.437937),
 (52.544616, 13.437842),
 (52.544644, 13.437763),
 (52.544727, 13.437512),
 (52.544928, 13.436946),
 (52.544962, 13.436846),
 (52.545003, 13.43673),
 (52.545148, 13.436313),
 (52.545492, 13.435323),
 (52.545845, 13.434292),
 (52.545879, 13.434194),
 (52.545913, 13.434094),
 (52.546224, 13.433211),
 (52.546256, 13.433117),
 (52.546287, 13.433031),
 (52.546565, 13.432231),
 (52.546588, 13.432164),
 (52.546612, 13.432092),
 (52.546897, 13.431268),
 (52.54692, 13.431198),
 (52.546863, 13.431142),
 (52.545815, 13.430129),
 (52.545776, 13.43009),
 (52.545807, 13.430018),
 (52.545962, 13.429661),
 (52.546002, 13.429562),
 (52.54601, 13.429398),
 (52.545436, 13.428696),
 (52.54538, 13.428631),
 (52.545636, 13.428054),
 (52.54

In [17]:
f = folium.Figure(width=700, height=500)
m = folium.Map(location=[52.5300, 13.4150], zoom_start=13).add_to(f)

folium.Marker(start_coord, tooltip=start_location).add_to(m)
folium.Marker(end_coord, tooltip=end_location).add_to(m)

# Add route to the map
folium.PolyLine(route_coords, color='blue', weight=5, opacity=0.8).add_to(m)

m